In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

C:\Users\mdsmb\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
calender_df = pd.read_csv('calendar.csv')
calender_df

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,NaN,0,0,0,0,Frankfurt_1
1,2020-03-22,NaN,0,0,0,0,Frankfurt_1
2,2018-02-07,NaN,0,0,0,0,Frankfurt_1
3,2018-08-10,NaN,0,0,0,0,Frankfurt_1
4,2017-10-26,NaN,0,0,0,0,Prague_2
...,...,...,...,...,...,...,...
23011,2019-04-20,NaN,1,0,0,0,Munich_1
23012,2018-03-31,NaN,1,0,0,0,Munich_1
23013,2017-04-15,NaN,1,0,0,0,Munich_1
23014,2023-04-08,NaN,1,0,0,1,Munich_1


- **1.warehouse** - warehouse name
- **2.date** - date
- **3.holiday_name** - name of public holiday if any
- **4.holiday** - 0/1 indicating the presence of holidays
- **5.shops_closed** - public holiday with most of the shops or large part of shops closed
- **6.winter_school_holidays** - winter school holidays
- **7.school_holidays** - school holidays

In [3]:
inventory_df = pd.read_csv('inventory.csv')
inventory_df

,unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
0,5255,2583,Pastry_196,Bakery,Bakery_L2_14,Bakery_L3_26,Bakery_L4_1,Prague_3
1,4948,2426,Herb_19,Fruit and vegetable,Fruit and vegetable_L2_30,Fruit and vegetable_L3_86,Fruit and vegetable_L4_1,Prague_3
2,2146,1079,Beet_2,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_65,Fruit and vegetable_L4_34,Prague_1
3,501,260,Chicken_13,Meat and fish,Meat and fish_L2_13,Meat and fish_L3_27,Meat and fish_L4_5,Prague_1
4,4461,2197,Chicory_1,Fruit and vegetable,Fruit and vegetable_L2_17,Fruit and vegetable_L3_33,Fruit and vegetable_L4_1,Frankfurt_1
...,...,...,...,...,...,...,...,...
5427,3772,1849,Bread_147,Bakery,Bakery_L2_9,Bakery_L3_146,Bakery_L4_1,Budapest_1
5428,5191,2555,Pastry_195,Bakery,Bakery_L2_14,Bakery_L3_26,Bakery_L4_1,Prague_2
5429,5226,2570,Pumpkin_22,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_99,Fruit and vegetable_L4_1,Prague_3
5430,5390,2649,Orange_32,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_46,Fruit and vegetable_L4_13,Budapest_1


- **1.unique_id** - inventory id for a single keeping unit
- **2.product_unique_id** - product id, inventory in each warehouse has the same product unique id (same products across all warehouses has the same product id, but different unique id)
- **3.name** - inventory id for a single keeping unit
- **4.L1_category_name, L2_category_name,** … - name of the internal category, the higher the number, the more granular information is present
- **5.warehouse** - warehouse name

In [4]:
sales_test_df = pd.read_csv('sales_test.csv')
sales_test_df

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
47016,4572,2024-06-03,Munich_1,5254.0,2.09,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
47017,3735,2024-06-04,Prague_1,9698.0,11.00,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
47018,3735,2024-06-03,Prague_1,10256.0,11.00,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
47019,2129,2024-06-03,Brno_1,8679.0,37.75,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
sales_train_df = pd.read_csv('sales_train.csv')
sales_train_df

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007414,4941,2023-06-21,Prague_1,9988.0,26.56,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
4007415,4941,2023-06-24,Prague_1,8518.0,27.42,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
4007416,4941,2023-06-23,Prague_1,10424.0,33.39,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
4007417,4941,2023-06-22,Prague_1,10342.0,22.88,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0


- **1.unique_id** - unique id for inventory
- **2.date** - date
- **3.warehouse** - warehouse name
- **4.total_orders** - historical orders for selected Rohlik warehouse known also for test set as a part of this challenge
- **5.sales** - Target value, sales volume (either in pcs or kg) adjusted by availability. The sales with lower availability than 1 are already adjusted to full potential sales by Rohlik internal logic. There might be missing dates both in train and test for a given inventory due to various reasons. This column is missing in test.csv as it is target variable.
- **6.sell_price_main** - sell price
- **7.availability** - proportion of the day that the inventory was available to customers. The inventory doesn't need to be available at all times. A value of 1 means it was available for the entire day. This column is missing in test.csv as it is not known at the moment of making the prediction.
- **8.type_0_discount, type_1_discount**, … - Rohlik is running different types of promo sale actions, these show the percentage of the original price during promo ((original price - current_price) / original_price). Multiple discounts with different type can be run at the same time, but always the highest possible discount among these types is used for sales. Negative discount value should be interpreted as no discount.

In [6]:
solutions_df = pd.read_csv('solution.csv')   #just an example df for submission
solutions_df

,id,sales_hat
0,1226_2024-06-03,0
1,1226_2024-06-11,0
2,1226_2024-06-13,0
3,1226_2024-06-15,0
4,1226_2024-06-09,0
...,...,...
47016,4572_2024-06-03,0
47017,3735_2024-06-04,0
47018,3735_2024-06-03,0
47019,2129_2024-06-03,0


###### - This is the format for your submission file

In [7]:
test_weights_df = pd.read_csv('test_weights.csv')
test_weights_df

,unique_id,weight
0,0,2.535369
1,1,3.888933
2,2,5.885237
3,3,2.139552
4,5,3.021715
...,...,...
5385,5427,1.690246
5386,5428,9.003782
5387,5429,14.505492
5388,5430,6.964847


### Merging Sales_train with inventory and calender

In [8]:
sales_train_merged = pd.merge(sales_train_df,inventory_df,on=['unique_id','warehouse'], how ='left')

In [9]:
sales_train_merged

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007414,4941,2023-06-21,Prague_1,9988.0,26.56,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,2422,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1
4007415,4941,2023-06-24,Prague_1,8518.0,27.42,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,2422,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1
4007416,4941,2023-06-23,Prague_1,10424.0,33.39,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,2422,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1
4007417,4941,2023-06-22,Prague_1,10342.0,22.88,34.06,1.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,2422,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1


In [10]:
sales_train_full = pd.merge(sales_train_merged, calender_df,on =['date','warehouse'], how = 'left')

In [11]:
sales_train_full

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,NaN,0,0,0,0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,NaN,0,0,0,0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,NaN,0,0,0,0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,NaN,0,0,0,0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007414,4941,2023-06-21,Prague_1,9988.0,26.56,34.06,1.00,0.00000,0.0,0.0,...,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,NaN,0,0,0,0
4007415,4941,2023-06-24,Prague_1,8518.0,27.42,34.06,1.00,0.00000,0.0,0.0,...,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,NaN,0,0,0,0
4007416,4941,2023-06-23,Prague_1,10424.0,33.39,34.06,1.00,0.00000,0.0,0.0,...,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,NaN,0,0,0,0
4007417,4941,2023-06-22,Prague_1,10342.0,22.88,34.06,1.00,0.00000,0.0,0.0,...,Kohlrabi_9,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_114,Fruit and vegetable_L4_1,NaN,0,0,0,0


### Now, Merging Sales test df with inventory df,calender_df

In [12]:
sales_test_merged = pd.merge(sales_test_df,inventory_df,on=['unique_id','warehouse'], how ='left')

In [13]:
sales_test_full = pd.merge(sales_test_merged, calender_df,on =['date','warehouse'], how = 'left')

In [14]:
sales_test_full

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,...,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,NaN,0,0,0,0
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,NaN,0,0,0,0
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,NaN,0,0,0,0
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,NaN,0,0,0,0
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,...,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47016,4572,2024-06-03,Munich_1,5254.0,2.09,0.00000,0.0,0.0,0.0,0.0,...,Apple_123,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_31,Fruit and vegetable_L4_1,NaN,0,0,0,0
47017,3735,2024-06-04,Prague_1,9698.0,11.00,0.00000,0.0,0.0,0.0,0.0,...,Kiwi_18,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_39,Fruit and vegetable_L4_52,NaN,0,0,0,0
47018,3735,2024-06-03,Prague_1,10256.0,11.00,0.00000,0.0,0.0,0.0,0.0,...,Kiwi_18,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_39,Fruit and vegetable_L4_52,NaN,0,0,0,0
47019,2129,2024-06-03,Brno_1,8679.0,37.75,0.00000,0.0,0.0,0.0,0.0,...,Grape_15,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_12,Fruit and vegetable_L4_1,NaN,0,0,0,0


# Data Preprocessing

## Handling missing values

In [15]:
sales_train_full.isnull().sum()

unique_id                       0
date                            0
warehouse                       0
total_orders                   52
sales                          52
sell_price_main                 0
availability                    0
type_0_discount                 0
type_1_discount                 0
type_2_discount                 0
type_3_discount                 0
type_4_discount                 0
type_5_discount                 0
type_6_discount                 0
product_unique_id               0
name                            0
L1_category_name_en             0
L2_category_name_en             0
L3_category_name_en             0
L4_category_name_en             0
holiday_name              3844119
holiday                         0
shops_closed                    0
winter_school_holidays          0
school_holidays                 0
dtype: int64

## Total Orders

In [16]:
sales_train_full['total_orders'].fillna(0,inplace = True)

## Holiday Name

In [17]:
sales_train_full['holiday_name'].fillna('No Holiday', inplace = True)

# Missing values in target variable

In [18]:
missing_target_rows =sales_train_full.sales.isnull()

In [19]:
sales_train_full = sales_train_full[~missing_target_rows]

In [20]:
sales_train_full.sales.isnull().sum()

0

## missing values in test set

In [21]:
sales_test_full.isnull().sum()

unique_id                     0
date                          0
warehouse                     0
total_orders                  0
sell_price_main               0
type_0_discount               0
type_1_discount               0
type_2_discount               0
type_3_discount               0
type_4_discount               0
type_5_discount               0
type_6_discount               0
product_unique_id             0
name                          0
L1_category_name_en           0
L2_category_name_en           0
L3_category_name_en           0
L4_category_name_en           0
holiday_name              46544
holiday                       0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
dtype: int64

In [22]:
sales_test_full['holiday_name'].fillna('No Holiday', inplace = True)

## Checking Duplicates

In [23]:
sales_train_full.duplicated().sum()

0

In [24]:
sales_test_full.duplicated().sum()

0

###### - No duplicates

In [25]:
sales_train_full.shape

(4007367, 25)

## Interpreting negative discount as no discount as per REQUIREMENT

#### Type 0

In [26]:
sales_train_full[sales_train_full['type_0_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
668474,1123,2022-01-19,Budapest_1,5560.0,281.64,607.43,1.00,-0.42066,0.0,0.0,...,Cauliflower_0,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_23,Fruit and vegetable_L4_1,No Holiday,0,0,0,0
668581,1123,2022-01-20,Budapest_1,5827.0,232.73,607.43,1.00,-0.20402,0.0,0.0,...,Cauliflower_0,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_23,Fruit and vegetable_L4_1,No Holiday,0,0,0,0
2250867,2402,2021-09-14,Budapest_1,4740.0,24.61,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251021,2402,2021-09-12,Budapest_1,4614.0,19.29,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251031,2402,2021-09-08,Budapest_1,4739.0,41.54,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251112,2402,2021-09-29,Budapest_1,4903.0,21.93,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251176,2402,2021-10-04,Budapest_1,5094.0,34.69,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251480,2402,2021-09-11,Budapest_1,4539.0,29.75,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251540,2402,2021-09-30,Budapest_1,5135.0,46.68,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0
2251547,2402,2021-10-03,Budapest_1,4741.0,35.37,1065.89,1.00,-0.04032,0.0,0.0,...,Pork_21,Meat and fish,Meat and fish_L2_15,Meat and fish_L3_61,Meat and fish_L4_1,No Holiday,0,0,0,0


In [27]:
sales_train_full[sales_train_full['type_0_discount']<0] = 0

In [28]:
sales_test_full[sales_test_full['type_0_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### Type 1

In [29]:
sales_train_full[sales_train_full['type_1_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


In [30]:
sales_test_full[sales_test_full['type_1_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### type 2

In [31]:
sales_train_full[sales_train_full['type_2_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


In [32]:
sales_test_full[sales_test_full['type_2_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### type3

In [33]:
sales_train_full[sales_train_full['type_3_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


In [34]:
sales_test_full[sales_test_full['type_3_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### type 4

In [35]:
sales_train_full[sales_train_full['type_4_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
211,4845,2021-05-20,Budapest_1,5124.0,11.01,455.96,1.0,0.0,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,No Holiday,0,0,0,0


In [36]:
sales_train_full[sales_train_full['type_4_discount']<0] = 0

In [37]:
sales_test_full[sales_test_full['type_4_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### type 5

In [38]:
sales_train_full[sales_train_full['type_5_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


In [39]:
sales_test_full[sales_test_full['type_5_discount']<0] 

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


#### type 6

In [40]:
sales_train_full[sales_train_full['type_6_discount']<0] 

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays
2789972,4447,2022-03-11,Prague_2,5285.0,14.25,129.13,0.21,0.0,0.0,0.0,...,Chicken_86,Meat and fish,Meat and fish_L2_13,Meat and fish_L3_24,Meat and fish_L4_4,No Holiday,0,0,0,0
2790605,4447,2022-03-10,Prague_2,5260.0,3.27,129.20,0.91,0.0,0.0,0.0,...,Chicken_86,Meat and fish,Meat and fish_L2_13,Meat and fish_L3_24,Meat and fish_L4_4,No Holiday,0,0,0,0
2948477,5303,2022-02-01,Budapest_1,5804.0,3.44,7640.56,1.00,0.0,0.0,0.0,...,Salmon_19,Meat and fish,Meat and fish_L2_25,Meat and fish_L3_71,Meat and fish_L4_1,Memorial Day of the Republic,0,0,0,0
2949020,5303,2022-01-31,Budapest_1,5598.0,0.63,7513.00,1.00,0.0,0.0,0.0,...,Salmon_19,Meat and fish,Meat and fish_L2_25,Meat and fish_L3_71,Meat and fish_L4_1,No Holiday,0,0,0,0


In [41]:
sales_train_full[sales_train_full['type_6_discount']<0] = 0

In [42]:
sales_test_full[sales_test_full['type_6_discount']<0]

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays


# step 2

In [43]:
sales_train_full['max_discount'] = sales_train_full[['type_0_discount','type_1_discount','type_2_discount','type_3_discount',
                                                    'type_4_discount','type_5_discount','type_6_discount']].max(axis = 1)

In [44]:
sales_test_full['max_discount'] = sales_test_full[['type_0_discount','type_1_discount','type_2_discount','type_3_discount',
                                                    'type_4_discount','type_5_discount','type_6_discount']].max(axis = 1)

## step 3

In [45]:
sales_train_full['total_discount'] = sales_train_full[['type_0_discount','type_1_discount','type_2_discount','type_3_discount',
                                                    'type_4_discount','type_5_discount','type_6_discount']].sum(axis = 1)

In [46]:
sales_test_full['total_discount'] = sales_test_full[['type_0_discount','type_1_discount','type_2_discount','type_3_discount',
                                                    'type_4_discount','type_5_discount','type_6_discount']].sum(axis = 1)

In [47]:
sales_test_full = sales_test_full.sort_values(by="date")

# Feature Engineering

### Warehouse Sales

In [48]:
# Aggregate sales by warehouse and product
warehouse_sales = sales_train_full.groupby('warehouse')['sales'].mean().reset_index()
warehouse_sales.columns = ['warehouse', 'avg_warehouse_sales']
sales_train_full = sales_train_full.merge(warehouse_sales, on='warehouse', how='left')

In [49]:
sales_test_full = sales_test_full.merge(warehouse_sales, on='warehouse', how='left')

## Product Sales 

In [50]:
product_sales = sales_train_full.groupby('product_unique_id')['sales'].mean().reset_index()
product_sales.columns = ['product_unique_id', 'avg_product_sales']
sales_train_full = sales_train_full.merge(product_sales, on='product_unique_id', how='left')

In [51]:
sales_test_full = sales_test_full.merge(product_sales, on='product_unique_id', how='left')

## Interaction of discount

In [52]:
# Calculate interaction features for both train and test datasets
sales_train_full['discount_price'] = sales_train_full['sell_price_main'] * (1 - sales_train_full['max_discount'])
sales_train_full['total_discount_price'] = sales_train_full['sell_price_main'] * (1 - sales_train_full['total_discount'])

sales_test_full['discount_price'] = sales_test_full['sell_price_main'] * (1 - sales_test_full['max_discount'])
sales_test_full['total_discount_price'] = sales_test_full['sell_price_main'] * (1 - sales_test_full['total_discount'])

In [53]:
sales_test_full.fillna(0,inplace = True)

# Date

In [55]:
# Step 1: Convert to datetime format
sales_train_full['date'] = pd.to_datetime(sales_train_full['date'], errors='coerce')


#preserving original date
sales_train_full['date_str'] =sales_train_full['date'].dt.strftime('%Y-%m-%d')

# Step 2: Extract features
sales_train_full['day_of_week'] = sales_train_full['date'].dt.dayofweek   # Monday=0, Sunday=6
sales_train_full['month'] = sales_train_full['date'].dt.month              # 1 to 12
sales_train_full['day_of_year'] = sales_train_full['date'].dt.dayofyear    # 1 to 366
sales_train_full['quarter'] = sales_train_full['date'].dt.quarter
sales_train_full['is_weekend'] = sales_train_full['date'].dt.weekday.isin([5, 6]).astype(int)  


#cyclic encoding
sales_train_full['sin_day_of_week'] = np.sin(2 * np.pi * sales_train_full['day_of_week']/7)
sales_train_full['cos_day_of_week'] = np.cos(2 * np.pi * sales_train_full['day_of_week']/7)
sales_train_full['sin_month'] = np.sin(2 * np.pi * sales_train_full['month']/12)
sales_train_full['cos_month'] = np.cos(2 * np.pi * sales_train_full['month']/12)

sales_train_full["day"] = sales_train_full["date"].dt.day

sales_train_full['day_sin'] = np.sin(2 * np.pi * sales_train_full['day'] / 31)  
sales_train_full['day_cos'] = np.cos(2 * np.pi * sales_train_full['day'] / 31)

# Apply sine and cosine transformation
sales_train_full["sin_dayofyear"] = np.sin(2 * np.pi * sales_train_full["day_of_year"] / 365)
sales_train_full["cos_dayofyear"] = np.cos(2 * np.pi * sales_train_full["day_of_year"] / 365)


# Check the new features
sales_train_full.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,is_weekend,sin_day_of_week,cos_day_of_week,sin_month,cos_month,day,day_sin,day_cos,sin_dayofyear,cos_dayofyear
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,1,-0.781831,0.623490,1.000000e+00,6.123234e-17,10.0,0.897805,-0.440394,0.933837,0.357698
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,0,0.781831,0.623490,5.000000e-01,-8.660254e-01,25.0,-0.937752,0.347305,0.601624,-0.798779
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,0,0.000000,1.000000,-2.449294e-16,1.000000e+00,20.0,-0.790776,-0.612106,-0.188227,0.982126
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,1,-0.974928,-0.222521,8.660254e-01,-5.000000e-01,29.0,-0.394356,0.918958,0.888057,-0.459733
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,0,-0.433884,-0.900969,8.660254e-01,-5.000000e-01,1.0,0.201299,0.979530,0.999991,0.004304


In [56]:
# Step 1: Convert to datetime format
sales_test_full['date'] = pd.to_datetime(sales_test_full['date'], errors='coerce')

#preserving original date
sales_test_full['date_str'] = sales_test_full['date'].dt.strftime('%Y-%m-%d')


# Step 2: Extract features
sales_test_full['day_of_week'] = sales_test_full['date'].dt.dayofweek   # Monday=0, Sunday=6
sales_test_full['month'] = sales_test_full['date'].dt.month              # 1 to 12
sales_test_full['day_of_year'] = sales_test_full['date'].dt.dayofyear    # 1 to 366
sales_test_full['quarter'] = sales_test_full['date'].dt.quarter
sales_test_full['is_weekend'] = sales_test_full['date'].dt.weekday.isin([5, 6]).astype(int)  # 1 if weekend, 0 if weekday

#cyclic encoding
sales_test_full['sin_day_of_week'] = np.sin(2 * np.pi * sales_test_full['day_of_week']/7)
sales_test_full['cos_day_of_week'] = np.cos(2 * np.pi * sales_test_full['day_of_week']/7)
sales_test_full['sin_month'] = np.sin(2 * np.pi * sales_test_full['month']/12)
sales_test_full['cos_month'] = np.cos(2 * np.pi * sales_test_full['month']/12)

sales_test_full["day"] = sales_test_full["date"].dt.day


sales_test_full['day_sin'] = np.sin(2 * np.pi * sales_test_full['day'] / 31)  
sales_test_full['day_cos'] = np.cos(2 * np.pi * sales_test_full['day'] / 31)

# Apply sine and cosine transformation
sales_test_full["sin_dayofyear"] = np.sin(2 * np.pi * sales_test_full["day_of_year"] / 365)
sales_test_full["cos_dayofyear"] = np.cos(2 * np.pi * sales_test_full["day_of_year"] / 365)


# Check the new features
sales_test_full.head()

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,is_weekend,sin_day_of_week,cos_day_of_week,sin_month,cos_month,day,day_sin,day_cos,sin_dayofyear,cos_dayofyear
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.224647e-16,-1.0,3,0.571268,0.820763,0.455907,-0.890028
1,3510,2024-06-03,Prague_2,5705.0,39.71,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.224647e-16,-1.0,3,0.571268,0.820763,0.455907,-0.890028
2,3517,2024-06-03,Prague_3,5209.0,67.55,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.224647e-16,-1.0,3,0.571268,0.820763,0.455907,-0.890028
3,3148,2024-06-03,Prague_2,5705.0,53.04,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.224647e-16,-1.0,3,0.571268,0.820763,0.455907,-0.890028
4,2385,2024-06-03,Brno_1,8679.0,67.14,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.224647e-16,-1.0,3,0.571268,0.820763,0.455907,-0.890028


In [57]:
sales_train_full.drop('availability', axis =1,inplace = True)

In [58]:
# Calculate weekend discount for both train and test datasets
sales_train_full['weekend_discount'] = sales_train_full['is_weekend'] * sales_train_full['total_discount']
sales_test_full['weekend_discount'] = sales_test_full['is_weekend'] * sales_test_full['total_discount']

In [59]:
sales_test_full.fillna(0,inplace = True)

In [60]:
x_train_with_id = sales_train_full[['unique_id','date_str']]

In [61]:
x_test_with_id = sales_test_full[['unique_id','date_str']]

In [62]:
sales_train_full['date'].min()

Timestamp('2020-08-01 00:00:00')

In [63]:
sales_train_full['date'].max()

Timestamp('2024-06-02 00:00:00')

In [64]:
sales_test_full['date'].min()

Timestamp('2024-06-03 00:00:00')

In [65]:
sales_test_full['date'].max()

Timestamp('2024-06-16 00:00:00')

#  data splitting

In [66]:
train_start_date = '2023-08-01'
train_end_date = '2024-06-02'

#val_start_date = '2024-05-20'
#val_end_date = '2024-06-02'

test_start_date = '2024-06-03'
test_end_date = '2024-06-16'

# Convert date column to datetime format
sales_train_full['date'] = pd.to_datetime(sales_train_full['date'])
sales_test_full['date'] = pd.to_datetime(sales_test_full['date'])

# ✅ Train Dataset (Only past data)
train = sales_train_full[(sales_train_full['date'] >= train_start_date) & (sales_train_full['date'] <= train_end_date)]

# ✅ Validation Dataset (Recent unseen data)
#val = sales_train_full[(sales_train_full['date'] >= val_start_date) & (sales_train_full['date'] <= val_end_date)]

# ✅ Test Dataset (Future data for prediction)
test = sales_test_full[(sales_test_full['date'] >= test_start_date) & (sales_test_full['date'] <= test_end_date)]

# Training the model

In [67]:
train.drop('date', axis =1,inplace = True)

In [68]:
test.drop('date', axis =1,inplace = True)

In [70]:
x_train = train.drop(['sales','date_str'], axis = 1)
y_train =train['sales']

In [71]:
x_test = test.drop(['date_str'], axis = 1)

In [72]:
x_train.shape

(1059802, 43)

In [73]:
y_train.shape

(1059802,)

In [74]:
x_test.shape

(47021, 43)

### Defining categorical features

In [75]:
cat_features = ['warehouse','name','L1_category_name_en','L2_category_name_en','L3_category_name_en','L4_category_name_en','holiday_name']

In [76]:
x_train[cat_features] = x_train[cat_features].astype('category')

In [78]:
x_test[cat_features] = x_test[cat_features].astype('category')

# LightGBm

In [79]:
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error

In [80]:
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# ✅ Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'rmse',
        'metric': 'mae',  # Minimize WMAE
        'boosting_type': 'gbdt',  # Gradient Boosting Decision Trees
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 500),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 10),
        'max_bin': trial.suggest_int('max_bin', 50, 300),
        'verbosity': -1,  # Suppress logs
    }
    
    # ✅ Perform Time Series Split inside Optuna
    tscv = TimeSeriesSplit(n_splits=3)  # 3-fold time-series cross-validation
    val_scores = []

    for train_index, val_index in tscv.split(x_train):
        X_train_small, X_val = x_train.iloc[train_index], x_train.iloc[val_index]
        y_train_small, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

        # ✅ Train LightGBM with Trial Parameters
        train_data = lgb.Dataset(X_train_small, label=y_train_small, categorical_feature=cat_features)
        val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=cat_features)

        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data]
        )

        # ✅ Predict on Validation Set
        preds = model.predict(X_val)
        val_scores.append(mean_absolute_error(y_val, preds))

    # ✅ Return the average MAE from cross-validation
    return sum(val_scores) / len(val_scores)

# 🚀 Run Optuna Optimization (100 Trials for Best Results)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)

# ✅ Get Best Hyperparameters
best_params = study.best_params
print("\n🚀 Best Hyperparameters Found:\n", best_params)

[I 2025-02-14 12:37:25,963] A new study created in memory with name: no-name-9f671116-9878-4e82-a087-81aeea587934
[I 2025-02-14 12:37:42,304] Trial 0 finished with value: 60.803300961927356 and parameters: {'learning_rate': 0.010284772497876656, 'num_leaves': 92, 'max_depth': 18, 'min_data_in_leaf': 393, 'feature_fraction': 0.45488126458368466, 'bagging_fraction': 0.7927914449741766, 'bagging_freq': 5, 'lambda_l1': 9.476918394768416e-06, 'lambda_l2': 0.1169367786184681, 'min_gain_to_split': 8.963064397402237, 'max_bin': 115}. Best is trial 0 with value: 60.803300961927356.
[I 2025-02-14 12:37:58,496] Trial 1 finished with value: 67.07912744781093 and parameters: {'learning_rate': 0.007398594798594104, 'num_leaves': 140, 'max_depth': 14, 'min_data_in_leaf': 64, 'feature_fraction': 0.9891781055756329, 'bagging_fraction': 0.7650984393437716, 'bagging_freq': 4, 'lambda_l1': 6.757230138222416e-08, 'lambda_l2': 1.8282041902963712e-06, 'min_gain_to_split': 9.209491662759671, 'max_bin': 124}. 

[I 2025-02-14 12:41:55,407] Trial 18 finished with value: 36.99986779583285 and parameters: {'learning_rate': 0.19285113486237612, 'num_leaves': 124, 'max_depth': 16, 'min_data_in_leaf': 17, 'feature_fraction': 0.6056651895736794, 'bagging_fraction': 0.5139496274930782, 'bagging_freq': 3, 'lambda_l1': 1.1725662833050433e-08, 'lambda_l2': 0.7732139915990961, 'min_gain_to_split': 2.2028247874335736, 'max_bin': 151}. Best is trial 15 with value: 34.885218852738056.
[I 2025-02-14 12:42:12,972] Trial 19 finished with value: 38.52995514750145 and parameters: {'learning_rate': 0.03238940660725932, 'num_leaves': 166, 'max_depth': 17, 'min_data_in_leaf': 131, 'feature_fraction': 0.533851083812096, 'bagging_fraction': 0.6327713769670993, 'bagging_freq': 3, 'lambda_l1': 5.304406579225237e-06, 'lambda_l2': 0.04332453314824699, 'min_gain_to_split': 4.668008641562354, 'max_bin': 220}. Best is trial 15 with value: 34.885218852738056.
[I 2025-02-14 12:42:27,626] Trial 20 finished with value: 43.406590

[I 2025-02-14 12:47:05,078] Trial 36 finished with value: 34.247352152900426 and parameters: {'learning_rate': 0.14919671490554617, 'num_leaves': 184, 'max_depth': 18, 'min_data_in_leaf': 41, 'feature_fraction': 0.7049414932928028, 'bagging_fraction': 0.7928823434395171, 'bagging_freq': 3, 'lambda_l1': 0.037032059720412555, 'lambda_l2': 0.01153245649470737, 'min_gain_to_split': 6.860817815831865, 'max_bin': 165}. Best is trial 36 with value: 34.247352152900426.
[I 2025-02-14 12:47:23,970] Trial 37 finished with value: 34.238806730777604 and parameters: {'learning_rate': 0.14969617728381887, 'num_leaves': 186, 'max_depth': 20, 'min_data_in_leaf': 44, 'feature_fraction': 0.7730232842507789, 'bagging_fraction': 0.8368077704044784, 'bagging_freq': 2, 'lambda_l1': 0.0378224456800411, 'lambda_l2': 0.019871324936940545, 'min_gain_to_split': 7.9517479619645615, 'max_bin': 135}. Best is trial 37 with value: 34.238806730777604.
[I 2025-02-14 12:47:38,713] Trial 38 finished with value: 34.2583755

[I 2025-02-14 12:51:49,888] Trial 54 finished with value: 35.74254540408185 and parameters: {'learning_rate': 0.13656620409327083, 'num_leaves': 191, 'max_depth': 19, 'min_data_in_leaf': 75, 'feature_fraction': 0.9129758142617596, 'bagging_fraction': 0.758373906546274, 'bagging_freq': 1, 'lambda_l1': 0.17876508661355178, 'lambda_l2': 6.351029516546623e-05, 'min_gain_to_split': 9.489457723101037, 'max_bin': 90}. Best is trial 48 with value: 33.53671198619035.
[I 2025-02-14 12:52:01,408] Trial 55 finished with value: 36.601369599432864 and parameters: {'learning_rate': 0.17279658312094856, 'num_leaves': 33, 'max_depth': 18, 'min_data_in_leaf': 113, 'feature_fraction': 0.8510511612263372, 'bagging_fraction': 0.7768478275885049, 'bagging_freq': 2, 'lambda_l1': 0.014094865825820995, 'lambda_l2': 8.273145314105202e-07, 'min_gain_to_split': 8.969866158375249, 'max_bin': 69}. Best is trial 48 with value: 33.53671198619035.
[I 2025-02-14 12:52:17,987] Trial 56 finished with value: 35.9107173781

[I 2025-02-14 12:57:03,068] Trial 72 finished with value: 33.917232945655506 and parameters: {'learning_rate': 0.10681968039275043, 'num_leaves': 162, 'max_depth': 19, 'min_data_in_leaf': 54, 'feature_fraction': 0.8024947706941081, 'bagging_fraction': 0.7108466196447157, 'bagging_freq': 2, 'lambda_l1': 0.5485625133282274, 'lambda_l2': 0.0011392244356855717, 'min_gain_to_split': 7.431627188917094, 'max_bin': 111}. Best is trial 48 with value: 33.53671198619035.
[I 2025-02-14 12:57:17,149] Trial 73 finished with value: 33.93460387350723 and parameters: {'learning_rate': 0.10588912934718006, 'num_leaves': 162, 'max_depth': 18, 'min_data_in_leaf': 22, 'feature_fraction': 0.851105572896001, 'bagging_fraction': 0.6601177950088305, 'bagging_freq': 1, 'lambda_l1': 0.5665976362194182, 'lambda_l2': 0.001219928505610302, 'min_gain_to_split': 7.270126612693628, 'max_bin': 117}. Best is trial 48 with value: 33.53671198619035.
[I 2025-02-14 12:57:35,113] Trial 74 finished with value: 35.673418909047

...

[I 2025-02-14 14:29:17,865] Trial 432 finished with value: 34.58736893516581 and parameters: {'learning_rate': 0.15993038096656062, 'num_leaves': 166, 'max_depth': 10, 'min_data_in_leaf': 43, 'feature_fraction': 0.8801729384823714, 'bagging_fraction': 0.8717545063471506, 'bagging_freq': 1, 'lambda_l1': 2.1269603163416793e-06, 'lambda_l2': 8.624443347362146e-07, 'min_gain_to_split': 8.018476006932303, 'max_bin': 67}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:29:31,283] Trial 433 finished with value: 33.86927880967638 and parameters: {'learning_rate': 0.14863377684660423, 'num_leaves': 164, 'max_depth': 11, 'min_data_in_leaf': 13, 'feature_fraction': 0.8691038352948667, 'bagging_fraction': 0.9218222423690364, 'bagging_freq': 1, 'lambda_l1': 4.811026407229358e-07, 'lambda_l2': 7.462116760521155e-07, 'min_gain_to_split': 8.203909815624376, 'max_bin': 90}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:29:46,752] Trial 434 finished with value: 33

[I 2025-02-14 14:33:30,617] Trial 450 finished with value: 33.123460937574535 and parameters: {'learning_rate': 0.19998749338000538, 'num_leaves': 176, 'max_depth': 11, 'min_data_in_leaf': 16, 'feature_fraction': 0.8737727893665787, 'bagging_fraction': 0.8166525241148174, 'bagging_freq': 1, 'lambda_l1': 2.2927836791857055e-07, 'lambda_l2': 5.062437497471871e-07, 'min_gain_to_split': 8.224171648036672, 'max_bin': 83}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:33:44,262] Trial 451 finished with value: 34.029802516169305 and parameters: {'learning_rate': 0.19727238533193792, 'num_leaves': 182, 'max_depth': 11, 'min_data_in_leaf': 39, 'feature_fraction': 0.9017711804915592, 'bagging_fraction': 0.8045828656853843, 'bagging_freq': 1, 'lambda_l1': 2.962694775606624e-07, 'lambda_l2': 1.8367001608558684e-07, 'min_gain_to_split': 8.521087514346716, 'max_bin': 80}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:33:57,720] Trial 452 finished with value:

[I 2025-02-14 14:37:45,407] Trial 468 finished with value: 34.344535756421216 and parameters: {'learning_rate': 0.14598144825299061, 'num_leaves': 154, 'max_depth': 11, 'min_data_in_leaf': 23, 'feature_fraction': 0.8728560011819616, 'bagging_fraction': 0.6239641389612651, 'bagging_freq': 2, 'lambda_l1': 7.051233098925413e-07, 'lambda_l2': 2.899419938207274e-06, 'min_gain_to_split': 8.230280600533838, 'max_bin': 95}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:37:58,617] Trial 469 finished with value: 33.321626759541296 and parameters: {'learning_rate': 0.16085410766427508, 'num_leaves': 170, 'max_depth': 10, 'min_data_in_leaf': 14, 'feature_fraction': 0.8330235917613182, 'bagging_fraction': 0.8705927025410074, 'bagging_freq': 1, 'lambda_l1': 5.545136163948934e-07, 'lambda_l2': 1.1954310477550282e-07, 'min_gain_to_split': 7.7856285682198605, 'max_bin': 80}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:38:12,037] Trial 470 finished with value:

[I 2025-02-14 14:41:36,237] Trial 486 finished with value: 33.28682652632235 and parameters: {'learning_rate': 0.1627399312931016, 'num_leaves': 150, 'max_depth': 10, 'min_data_in_leaf': 28, 'feature_fraction': 0.6091475025816959, 'bagging_fraction': 0.9634415615798462, 'bagging_freq': 1, 'lambda_l1': 4.995451588332786e-07, 'lambda_l2': 1.244367147817958e-06, 'min_gain_to_split': 8.725700312393453, 'max_bin': 82}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:41:49,395] Trial 487 finished with value: 33.286409378893445 and parameters: {'learning_rate': 0.18604401303891685, 'num_leaves': 165, 'max_depth': 11, 'min_data_in_leaf': 21, 'feature_fraction': 0.8653429191672839, 'bagging_fraction': 0.8833295835959564, 'bagging_freq': 1, 'lambda_l1': 2.6197889528468754e-06, 'lambda_l2': 8.093936733760493e-07, 'min_gain_to_split': 8.376164143297613, 'max_bin': 87}. Best is trial 322 with value: 32.874419779042846.
[I 2025-02-14 14:42:02,163] Trial 488 finished with value: 33


🚀 Best Hyperparameters Found:
 {'learning_rate': 0.17032587627692644, 'num_leaves': 160, 'max_depth': 11, 'min_data_in_leaf': 13, 'feature_fraction': 0.8920337603955795, 'bagging_fraction': 0.8494100258590612, 'bagging_freq': 2, 'lambda_l1': 2.38179891680092e-07, 'lambda_l2': 3.9377532525737197e-07, 'min_gain_to_split': 8.379769416175595, 'max_bin': 85}


In [81]:
# ✅ Train Final LightGBM Model with Best Params
lgb_model = lgb.train(
    best_params,
    lgb.Dataset(x_train, label=y_train, categorical_feature=cat_features),
    num_boost_round=2000,  # More rounds for better learning
)

# ✅ Make Final Predictions on Test Set (X_test)
x_test['sales_hat'] = lgb_model.predict(x_test)

In [82]:
x_test_with_id['sales_hat'] = x_test['sales_hat']

In [83]:
x_test_with_id = x_test_with_id.merge(test_weights_df, on='unique_id', how='left')

In [84]:
#Preparing submission file (for Kaggle)
x_test_with_id['id'] = x_test_with_id['unique_id'].astype(str) + "_" + x_test_with_id['date_str']  # Combine unique_id and date
submission_file = x_test_with_id[['id', 'sales_hat']]  # Select required columns for submission
submission_file.to_csv('submission_final.csv', index=False)  # Save as CSV file for submission